<a href="https://colab.research.google.com/github/k7sung/clap2choir/blob/master/sync.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# For syncing & rendering videos & audios

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Enter your authorization code:
··········
Mounted at /content/drive


In [2]:

GROUP = "Test" #@param ["Praise Team", "VHR", "Combined Choir", "Chamber Instrumental", "Test"] {type:"string", allow-input:true}
SONG = "Singing I Go" #@param ["Make My Life An Alleluia", "The Lord Is My Strength", "Singing I Go", "My Redeemer", "Sunshine In My Soul", "Day by Day", "I Will Exchange My Cross to Crown", "Hymn 157: Where Jesus Is, 'Tis Heaven", "Hymn 267: O For a Heart to Praise My God", "Hymn 420: Savior, Thy Dying Love", "Hymn 477: Then Jesus Came", "Hymn 479: O How He Loves You and Me", "Hymn 480: Wonderful, Wonderful Jesus", "Hymn 489: In My Heart There Rings a Melody", "Hymn 520: 'Twas Jesus' Blood", "Hymn 91: Thy Love, Jesus"] {allow-input: true}
DAY = 1 #@param {type:"integer"}
RUN_SCRIPT_TO = "check uploads" #@param ["check uploads", "the end"]

SHARED_FOLDER_PATH = 'Shared drives/MMP 2020 Participants' #@param {type:"string"}

OUTPUT_MODE = "mp3 only" #@param ["grid", "trimmed videos and audios", "mp3 only"]

VIDEO_RES = "1280x720" #@param ["1920x1080","1280x720", "854x480", "540x360"] 

SONG_NAME = f"{GROUP}/Day {DAY}/{SONG}"


### Optional Params

In [3]:
## OPTIONAL PARAMS ##
#TIME_ZONE = 'US/Central' #@param ["US/Central", "Europe/London", "Asia/Taipei"] {allow-input:true}
OUTPUT_SOLO = False #@param {type:"boolean"}
OUTPUT_COMPARE = True #@param {type:"boolean"}
OUTPUT_PARTS = False #@param {type:"boolean"}

VID_DOWNSCALE_RATIO =  4#@param {type:"integer"}

SUBTITLE_FILENAME = "" #@param {type:"string"}
RENDER_SUBTITLE = SUBTITLE_FILENAME != ""

KEEP_CLAPS = True #@param {type:"boolean"}
DEBUG_PLOT = False #@param {type:"boolean"}

NORMALIZE_VOLUME = True #@param {type:"boolean"}
MORE_ALIGNMENT = False #@param {type:"boolean"}

if OUTPUT_MODE == "trimmed videos and audios":
  OUTPUT_SOLO = True

## Mixer

In [4]:
VOL_S = 100 #@param {type:"slider", min:0, max:100}
VOL_A = 100 #@param {type:"slider", min:0, max:100}
VOL_T = 100 #@param {type:"slider", min:0, max:100}
VOL_B = 100 #@param {type:"slider", min:0, max:100}
VOL_M = 100 #@param {type:"slider", min:0, max:100}
VOL_OTHERS = 100 #@param {type:"slider", min:0, max:100}
# V_Violin1 = 100 #@param {type:"slider", min:0, max:100}
# V_Violin2 = 100 #@param {type:"slider", min:0, max:100}
# V_Viola = 100 #@param {type:"slider", min:0, max:100}
# V_Cello = 100 #@param {type:"slider", min:0, max:100}
# V_Bass = 100 #@param {type:"slider", min:0, max:100}
# V_Flute = 100 #@param {type:"slider", min:0, max:100}
# V_Clarinet = 100 #@param {type:"slider", min:0, max:100}
# V_Trumpet = 100 #@param {type:"slider", min:0, max:100}
# V_Trombone = 100 #@param {type:"slider", min:0, max:100}
# V_Cymbal = 100 #@param {type:"slider", min:0, max:100}
# V_Timpani = 100 #@param {type:"slider", min:0, max:100}
from collections import defaultdict
VOL = defaultdict(lambda:VOL_OTHERS, [
    ('S', VOL_S), ('A', VOL_A), ('T', VOL_T), ('B', VOL_B)
    # ,('VIOLIN1', V_Violin1),('VIOLIN2', V_Violin2), ('VIOLA', V_Viola), ('CELLO', V_Cello),('BASS', V_Bass),('FLUTE',V_Flute),('CLARINET', V_Clarinet), ('TRUMPET', V_Trumpet),("TROMBONE", V_Trombone),("CYMBALS",V_Cymbal),("TIMPANI",V_Timpani)
    ])

REVERB = 0 #@param {type: "slider", min:0, max:10}


## Just Codes

In [5]:
VID_FILE_GLOBS = ["*.mov", "*.MOV", "*.mp4", "*.MP4", "*.m4v"]
AUD_FILE_GLOBS = ["*.m4a", "*.mp3", "*.aac", "*.wav", "*.WAV"]


import numpy as np
from numpy import mean, median, std, var, diff, correlate as corr, zeros, transpose
from moviepy.editor import *
from moviepy.audio.AudioClip import AudioArrayClip

import glob
import os
from os.path import basename
import matplotlib.pyplot as plt
from itertools import combinations as comb
from random import shuffle, randint
import shutil
from collections import defaultdict

if SHARED_FOLDER_PATH:
    SHARED_DIR = "/content/drive/" + SHARED_FOLDER_PATH
    RAW_PATH = SHARED_DIR + f"/Inputs/{SONG_NAME}/"
    # OUTPUT_PATH = SHARED_DIR + f"/outputs/{SONG_NAME}/{TODAY_DATE}/"
    OUTPUT_PATH = SHARED_DIR + f"/Outputs/{SONG_NAME}/"



Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3661824/45929032 bytes (8.0%)7495680/45929032 bytes (16.3%)11485184/45929032 bytes (25.0%)15163392/45929032 bytes (33.0%)18579456/45929032 bytes (40.5%)21938176/45929032 bytes (47.8%)25804800/45929032 bytes (56.2%)29753344/45929032 bytes (64.8%)33628160/45929032 bytes (73.2%)37715968/45929032 bytes (82.1%)41598976/45929032 bytes (90.6%)45162496/45929032 bytes (98.3%)

In [6]:
from google.colab import drive

%mkdir -p "$OUTPUT_PATH"
if OUTPUT_COMPARE:
  %mkdir -p "$OUTPUT_PATH"compare
if OUTPUT_SOLO:
  %mkdir -p "$OUTPUT_PATH"solo
if OUTPUT_MODE == "trimmed videos and audios":
  %mkdir -p "$OUTPUT_PATH"vclips

In [7]:

def is_conductingvideo(filename):
    lbname = basename(filename).lower()
    return ('master' in lbname) or ('conducting' in lbname) or ('conductor' in lbname) or (lbname.startswith('m_'))


def get_loud_regular_patterns(tops, expected_n=4):
    ts = [t for t, v in tops]
    ls = [v for t, v in tops]
    candi_vars = [(var(diff([ts[i] for i in c])), c) for c in comb(range(len(ts)), expected_n)]
    candi_louds = [(sum([ls[i] for i in c]), c) for c in comb(range(len(ls)), expected_n)]
    max_var = max([c[0] for c in candi_vars])
    candi_vars = [(c[0] / max_var, c[1]) for c in candi_vars]
    max_loud = max([c[0] for c in candi_louds])
    candi_louds = [(-c[0] / max_loud, c[1]) for c in candi_louds]
    scores = [(1.0 * v[0] + 0.3 * l[0], v[1]) for v, l in zip(candi_vars, candi_louds)]
    most_regular = min(scores)
    #    print("most regular", most_regular)
    return [ts[t] for t in most_regular[1]]


def get_tops(peak_ts, peak_vals, d_peak_vals, top_n):
    tops = sorted([(t, 0.0 * v + 1.0 * dv) for t, v, dv in zip(peak_ts, peak_vals, d_peak_vals)], key=lambda k: k[1],
                  reverse=True)[:top_n]
    # print("tops: ", tops)
    return sorted(tops)


def estimate_start(audio, search_secs=20, is_finetune=False):
    c_dur = 0.1  # every chunk is 0.1 seconds
    chunks = [None] * int(search_secs / c_dur)

    # read the music in search range
    iter = audio.iter_chunks(chunk_duration=c_dur)
    for i in range(len(chunks)):
        chunk = iter.__next__()
        chunks[i] = chunk[:, 0]

    peaks = [(chunk.argmax(), chunk[chunk.argmax()]) for chunk in chunks]
    peak_vals = [p[1] for p in peaks]
    peak_ts = [(sec + p[0] * 1 / audio.fps) * c_dur for sec, p in zip(range(len(peaks)), peaks)]
    d_peaks = [0] + [t1 - t0 for t0, t1 in zip(peak_vals[:-1], peak_vals[1:])]

    is_master = False
    fname = basename(audio.filename)
    if is_conductingvideo(fname) and not is_finetune:
        print("this is is a conductor video")
        is_master = True

    if is_master:
        tops = get_tops(peak_ts, peak_vals, d_peaks, 16)
        clap_times = get_loud_regular_patterns(tops, 8)[-4:]
    else:
        if is_finetune:
            tops = get_tops(peak_ts, peak_vals, d_peaks, 8)
            clap_times = get_loud_regular_patterns(tops)
        else:
            tops = get_tops(peak_ts, peak_vals, d_peaks, 12)
            clap_times = get_loud_regular_patterns(tops)
            cheat_list = {  # "A_Lina_1.MOV":[4,5,6,7],
                # "T_Preston.mp4":[1,2,3,4] #,
                # "T_Enoch_1.mp4":[2,3,4,5]
            }
            if fname in cheat_list:
                sorted_t = sorted([t for t, v in tops])
                clap_times = [sorted_t[i] for i in cheat_list[fname]]
    est_intv = mean(diff(clap_times))
    start_est = mean(clap_times) + est_intv * 2
    if is_finetune == False:
        start_est = start_est - est_intv * 4.4
    # print("start est:", start_est)

    to_plot = lambda: plot_claps(fname, tops, peak_ts, peak_vals, d_peaks, clap_times)
    return start_est, est_intv, to_plot


def plot_claps(fname, tops, peak_ts, peak_vals, d_peaks, clap_times):
    print("\n" + fname + "\n")
    tops = np.array(tops)
    p1 = plt.subplot(211)
    plt.plot(peak_ts, peak_vals, '-', color="red")
    plt.plot(tops[:, 0], tops[:, 1], "x")
    p2 = plt.subplot(212)
    plt.plot(peak_ts, d_peaks, '-', color='blue')
    plt.plot(clap_times, [0] * len(clap_times), "x")
    plt.show()


def find_irregulars(ts):
    start_times = ts[0]
    est_beats = ts[1]
    to_plot_funcs = ts[2]

    beats_avg = mean(est_beats)
    beats_var = var(est_beats)
    irrs = []
    for b, plot in zip(est_beats, to_plot_funcs):
        if abs(b - beats_avg) > 0.15:
            print("noticeable irregular beats - expected:", (round(beats_avg, 3), round(std(est_beats), 3)),
                  "actual:", (round(b, 3), round(abs(b - beats_avg), 3)))
            if not DEBUG_PLOT:
                plot()
            irrs.append(True)
        elif (b - beats_avg) ** 2 > 5 * beats_var:  # print outliers
            print("maybe irregular beats - expected:", (round(beats_avg, 3), round(std(est_beats), 3)),
                  "actual:", (round(b, 3), round(abs(b - beats_avg), 3)))
            if not DEBUG_PLOT:
                plot()
            irrs.append(False)
        else:
            irrs.append(False)
        if DEBUG_PLOT:
            plot()
    return irrs

def estimate_start_x2(input_audios):
    if len(input_audios) == 0:
        return []
    print([basename(a.filename) for a in input_audios])
    t0s = [estimate_start(audio) for audio in input_audios]
    t0s = list(zip(*t0s))  # transpose t0s. t0s[0] is array of n elements of start_time
    print("____________first round______________")
    find_irregulars(t0s)
    t1s = [estimate_start(audio.subclip(t_start=max(st, 0)), search_secs=6, is_finetune=True)
           for audio, st in zip(input_audios, t0s[0])]
    t1s = list(zip(*t1s))
    print("____________second round_____________")
    bads = find_irregulars(t1s)
    return [max(max(0, t0) + t1, 0) for t0, t1 in zip(t0s[0], t1s[0])], bads


def get_files(dir, glob_pattern):
    return [filename for pattern in glob_pattern
            for filename in glob.glob(dir + pattern)
            if not basename(filename).startswith('_') and
            not basename(filename).startswith('.') and
            not 'delete' in basename(filename).lower()]



## Sync

In [ ]:

def get_conductor_video():
    # print(get_files(RAW_PATH, VID_FILE_GLOBS))
    cond_vids = [VideoFileClip(filename)
                 for filename in get_files(RAW_PATH, VID_FILE_GLOBS)
                 if is_conductingvideo(filename)]

    if cond_vids:
        cond_vid = cond_vids[0]
        cond_aud = cond_vid.audio
        cond_start_times, bads = estimate_start_x2([cond_aud])
        cond_start_time = cond_start_times[0]
        print("vid start:", cond_start_time)
        return cond_vid, cond_start_time

    cond_auds = [AudioFileClip(filename)
                 for filename in get_files(RAW_PATH, AUD_FILE_GLOBS)
                 if is_conductingvideo(filename)]
    if cond_auds:
        cond_aud = cond_auds[0]
        cond_start_times, bads = estimate_start_x2([cond_aud])
        cond_start_time = cond_start_times[0]
        print("aud start:", cond_start_time)
        return cond_aud, cond_start_time
    print("no conductor video found")
    return None, 5

cond_vid, cond_start_time = get_conductor_video()


In [ ]:

def align_files():
    input_vids = []
    for filename in get_files(RAW_PATH, VID_FILE_GLOBS):
        if is_conductingvideo(filename):
            continue
        try:
            v = VideoFileClip(filename)
            print(basename(v.filename))
            input_vids.append(v)
        except (IndexError, OSError) as e:
            print("*********** Cannot read **********", basename(filename))
            continue

    if input_vids:
        vid_start_times, vids_unaligned = estimate_start_x2([v.audio for v in input_vids])
    else:
        vid_start_times = []
        vids_unaligned = []

    if True in vids_unaligned:
        print("irregular claps detected:")
        for i, is_bad in zip(input_vids, vids_unaligned):
            if is_bad:
                print(basename(i.filename))

    input_auds = [AudioFileClip(filename) for filename in get_files(RAW_PATH, AUD_FILE_GLOBS)]
    if len(input_auds) > 0:
        aud_start_times, auds_unaligned = estimate_start_x2(input_auds)
    else:
        aud_start_times = []
        auds_unaligned = []

    if True in auds_unaligned:
        print("irregular claps detected:")
        for i, is_bad in zip(input_auds, auds_unaligned):
            if is_bad:
                print(basename(i.filename))
    return input_vids, vid_start_times, vids_unaligned, input_auds, aud_start_times, auds_unaligned


input_vids, vid_start_times, vids_unaligned, input_auds, aud_start_times, auds_unaligned  = align_files()


### Write synced files

In [10]:

def transform_outputs(cond_vid, cond_start_time, input_vids, vid_start_times, vids_unaligned, input_auds,
                      aud_start_times, auds_unaligned):
    output_vids = []
    output_auds = []
    if KEEP_CLAPS:
        output_vids = [(v.set_start(cond_start_time - st)) for v, st in zip(input_vids, vid_start_times)]
        output_auds = [(a.set_start(cond_start_time - st)) for a, st in zip(input_auds, aud_start_times)]
    else:
        output_vids = [(v.subclip(t_start=st)
                        .set_start(cond_start_time)) for v, st in zip(input_vids, vid_start_times)]
        output_auds = [(a.subclip(t_start=st)
                        .set_start(cond_start_time)) for a, st in zip(input_auds, aud_start_times)]

    if cond_vid and isinstance(cond_vid, VideoClip):
        if not 'invisible' in basename(cond_vid.filename):
            output_vids = output_vids + [cond_vid]
            output_auds = output_auds + [v.audio for v in output_vids]
        else:
            output_auds = output_auds + [v.audio for v in output_vids] + [cond_vid.audio]
    elif cond_vid and isinstance(cond_vid, AudioClip):
        output_auds = output_auds + [v.audio for v in output_vids] + [cond_vid]
    else:
        output_auds = output_auds + [v.audio for v in output_vids]

    aud_start_times = aud_start_times + vid_start_times + [cond_start_time]
    if NORMALIZE_VOLUME:
        output_auds = normalize_audios(output_auds, aud_start_times)
    output_vids = [v.set_audio(None) for v in output_vids]
    return output_vids, output_auds


def rms(audio):
    a_arr = audio.to_soundarray(fps=44100)
    return np.sqrt(np.mean(a_arr ** 2))


# if RUN_SCRIPT_TO == "the end":
def normalize_audios(output_auds, start_times=None):
    if not start_times:
        output_auds = [a.subclip(t_start=st) for a, st in zip(output_auds, start_times)]
    peak_volumes = [a.max_volume() for a in output_auds]
    rms_volumes = [rms(a) for a in output_auds]
    rms_pnorm_volumes = [rms / p for rms, p in zip(rms_volumes, peak_volumes)]  # rms of audio after normalized
    v_min = min(rms_pnorm_volumes)  # the rms of the quietest normalized audio

    # reduce everyone's rms to within 120% of the quietest audio
    target_vol_scales = [1 - (1 - v_min / v) * 0.8 for v in rms_pnorm_volumes]
    output_auds = [a.volumex(tv / p) for a, p, tv in zip(output_auds, peak_volumes, target_vol_scales)]
    return output_auds


if RUN_SCRIPT_TO == "the end":
    output_vids, output_auds = transform_outputs(cond_vid, cond_start_time, input_vids, vid_start_times, vids_unaligned, input_auds, aud_start_times, auds_unaligned)



In [11]:

def is_muted(audio_filename):
    if "muted" in basename(audio_filename).lower():
        print("muted ", basename(audio_filename))
        return True
    return False


def get_part(audio):
    if is_conductingvideo(audio.filename):
        return 'M'
    if is_muted(audio.filename):
        return 'Z'
    bname = basename(audio.filename)
    if '_' in bname:
        return bname.split('_')[0].upper()
    return bname[0].upper()


def get_part_vol(part, output_auds):
    if part == 'Z':
        print("mute volume at ", VOL[part])
    return VOL[part] / 100 / log(len(output_auds))


from math import log


def write_combined_audio_tracks(output_auds):
    VOL["Z"] = 0
    aud_groups = defaultdict(list)
    for a in output_auds:
        aud_groups[get_part(a)].append(a)

    aud_parts = [(part,
                  CompositeAudioClip(auds)
                  .volumex(get_part_vol(part, output_auds)))
                 for part, auds in aud_groups.items()]

    if OUTPUT_PARTS:
        for part, aud in aud_parts:
            aud.write_audiofile(OUTPUT_PATH + f"output_{part}.mp3", fps=44100, codec='mp3',
                                ffmpeg_params=['-aq', '2'])

    cc = CompositeAudioClip([a for p, a in aud_parts])
    cc.write_audiofile(OUTPUT_PATH + "output.mp3", fps=44100, codec='mp3', ffmpeg_params=['-aq', '2'])


if RUN_SCRIPT_TO == "the end":
    write_combined_audio_tracks(output_auds)

# ipython_display(cc, fps=44100, maxduration=360)


### Write extracted solo audio clips

In [12]:

def write_individual_clips(output_vids, output_auds):
    vFx = lambda v: (v.resize(height=H / VID_DOWNSCALE_RATIO)
                     )

    if OUTPUT_MODE == "trimmed videos and audios":
        for v in output_vids:
            vo = vFx(v)
            vo.write_videofile(OUTPUT_PATH + "vclips/" + basename(v.filename), fps=30, codec='libx264',
                               ffmpeg_params=['-crf', '18'])

    if OUTPUT_SOLO:
        for a in output_auds:
            b = CompositeAudioClip([a]).set_fps(44100)
            b.write_audiofile(OUTPUT_PATH + "solo/" + basename(a.filename) + ".mp3", fps=44100, codec='mp3',
                              ffmpeg_params=['-aq', '2'])

if RUN_SCRIPT_TO == "the end":
  write_individual_clips(output_vids, output_auds)


### Write compare clips

In [13]:

def write_compare_clips(output_auds):
    if OUTPUT_COMPARE:
        cc = AudioFileClip(OUTPUT_PATH + "output.mp3")
        cc_wavarray = cc.to_soundarray(fps=44100)[:, 0]
        for aud in output_auds:
            fname = basename(aud.filename)
            if OUTPUT_SOLO:
                aud = AudioFileClip(OUTPUT_PATH + "solo/" + fname + ".mp3").to_soundarray()[:, 0]
            else:
                aud = CompositeAudioClip([aud]).set_fps(44100).to_soundarray()[:, 0]
            mlen = min(len(aud), len(cc_wavarray))
            mix = AudioArrayClip(np.transpose([aud[:mlen], cc_wavarray[:mlen]-aud[:mlen]]), fps=44100)
            mix.write_audiofile(OUTPUT_PATH + "compare/" + fname + "_compare.mp3", codec='mp3')

if RUN_SCRIPT_TO == "the end":
    write_compare_clips(output_auds)

### Create video

In [14]:

def SATB_order(v):
    key = ''
    if 'filename' in v.__dict__:
        filepath = v.filename
        filename = basename(filepath)
        key = filename[0].lower()
    m = {'s': 0, 'a': 1, 't': 2, 'b': 3}
    if key in m.keys():
        return m[key]
    return randint(0, 3)


def get_vid_array_dim(n):
    i = 0
    while True:
        i += 1
        if i * i >= n:
            break

    if i * i == n:
        return (i, i)
    if (i - 1) * i >= n:
        return (i - 1, i)
    return (i, i)


def rotate_vid(vid):
    if vid.rotation in [-90, 90, 270]:
        vid = vid.resize(vid.size[::-1])
        vid.rotation = 0
    return vid


def get_manual_row_col(filename):
    fname = basename(filename)
    r, c = 0, 0
    if len(fname) >= 4:
        if fname[0].upper() == 'R' and fname[2].upper() == 'C' and \
                fname[1] in '123456789' and fname[3] in '123456789':
            r = int(fname[1])
            c = int(fname[3])
    return r, c


def is_manual_layout(vids):
    maxR = 0
    maxC = 0
    for v in vids:
        r, c = get_manual_row_col(v.filename)
        if r == 0 or c == 0:
            maxR, maxC = 0, 0
            break
        maxR = max(maxR, r)
        maxC = max(maxC, c)

    if maxR == 0 or maxC == 0:
        print("*** layout: automatic ***")
        r, c = get_vid_array_dim(len(vids))
        return False, r, c
    print("*** layout: manual ***")
    return True, maxR, maxC


def generate_grid_video(output_vids, output_auds):
    W, H = [int(t) for t in VIDEO_RES.split("x")]
    if OUTPUT_MODE == "grid":
        sub_height = H // 10
        if RENDER_SUBTITLE:
            H -= sub_height

        out_vids = output_vids
        out_vids = [rotate_vid(v) for v in out_vids]
        is_manual, ROWS, COLS = is_manual_layout(out_vids)
        out_vids = [v.resize(height=H // ROWS) for v in out_vids]
        out_vids = [v.crop(x_center=v.w // 2, width=min(W // COLS, v.w)) for v in out_vids]
        clip_duration = max([i.duration for i in out_vids])
        empty_clip = ColorClip((W // COLS, H // ROWS), (0, 0, 0), duration=clip_duration)
        if is_manual:
            tiles = np.full((ROWS, COLS), empty_clip)
            for v in out_vids:
                r, c = get_manual_row_col(v.filename)
                tiles[r - 1, c - 1] = v
        else:
            out_vids = out_vids + [empty_clip] * (ROWS * COLS - len(out_vids))
            shuffle(out_vids)
            out_vids.sort(key=SATB_order)
            tiles = np.reshape(out_vids, (COLS, ROWS))  # dimension is inversed because need to transpose in next line
            tiles = np.transpose(tiles)

        if RENDER_SUBTITLE:
            subspace_clip = ColorClip((W // COLS, sub_height), (0, 0, 0), duration=clip_duration)
            tiles = np.vstack([tiles, [subspace_clip] * COLS])

        cc = clips_array(tiles, cols_widths=[W // COLS] * COLS)
        if RENDER_SUBTITLE:
            cc = clips_array(tiles, cols_widths=[W // COLS] * COLS)
        cc = cc.set_audio(AudioFileClip(OUTPUT_PATH + "output.mp3"))

        # cc.save_frame("preview.png", t=10)
        # cc=cc.subclip(40,60)
        if H < 720:  # draft quality, saves time
            f_params = []
        else:
            f_params = ['-crf', '18']

        output_i = 0
        while os.path.exists(OUTPUT_PATH + f"output{output_i}.mp4"):
            output_i += 1
        cc.write_videofile(OUTPUT_PATH + f"output{output_i}.mp4", fps=30, codec='libx264', ffmpeg_params=f_params,
                           threads=4)


if RUN_SCRIPT_TO == "the end":
    generate_grid_video(output_vids, output_auds)
